<a href="https://colab.research.google.com/github/ricyoung/cs689/blob/master/HW3%20working%20on%20NMIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


**simple LR on Mnist**

In [0]:
#retrive data
mnist = pd.read_csv('https://mkang.faculty.unlv.edu/teaching/CS489_689/HW3/MNIST_15_15.csv', header = None)
mnist_label = pd.read_csv('https://mkang.faculty.unlv.edu/teaching/CS489_689/HW3/MNIST_LABEL.csv', header = None)
train_data = mnist
test_data = mnist_label

In [0]:
# train_data

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(train_data)
# rescaledX

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
#shuffle the data index for extracting the data later
data_index = np.arange(rescaledX.shape[0])
np.random.shuffle(data_index)

In [6]:
n, p = rescaledX.shape
n,p 

(335, 225)

In [0]:
# we convert a regression problem to a classification problem (discretization)
y = np.zeros(n)
y[train_data.iloc[:, -1] > 23] = 1

# data preparation for training data
X = train_data.iloc[:, 0:-1]
X = pd.DataFrame(np.c_[np.ones(n), X])

# from sklearn.model_selection import KFold
# X = mnist
# y = mnist_label
# kf = KFold(n_splits=10)
# kf.get_n_splits(X)
# # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
# print(kf)


# data preparation for test data
y_groundtruth = np.zeros(test_data.shape[0])
y_groundtruth[test_data.iloc[:, -1] > 23] = 1

X_test = test_data.iloc[:, 0:-1]
X_test = pd.DataFrame(np.c_[np.ones(n), X_test])

data_index = np.arange(mnist.shape[0])
np.random.shuffle(data_index)

##

In [0]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0, 1))
# rescaledX = scaler.fit_transform(train_data)
# rescaledX

In [0]:
#k-fold - split data index into 10parts
batch = []
for i in range (10):
    batch.append([])
    for j in range (len(data_index)):
        if j%10 == i:
            batch[i].append(data_index[j])



In [15]:
#loop for each batch

for i in range(len(batch)):
    #X is the train data, y is the labels
    X = train_data.__deepcopy__()
    y = test_data.__deepcopy__()
    #drop the selected set of data index(test data) from X and add a column of intercept
    X = np.array(X.drop(batch[i]))
    ones = np.ones((X.shape[0],1))
    X = np.hstack((X,ones))
    #drop the selected set of data index(test data) from the label, also change label, 5 to -1, 6 to 1
    y = np.array(y.drop(batch[i]))*2-11
    std = np.std(X,axis = 1)

    ###standard deviation normalization by column
    std =np.std(X, axis = 0)
    mean = np.mean(X, axis = 0)
    for j in range(X.shape[0]):
        for k in range(X.shape[1]):
            if std[k] > 0:
                X[j][k] = (X[j][k]-mean[k])/std[k]

    #find the coefficient (Kang code)
    b = ((np.linalg.inv(np.transpose(X).dot(X)+0.1*np.identity(X.shape[1]))).dot(np.transpose(X))).dot(y)

    #retrieve the test data
    test_X =[]
    test_y =[]
    for j in range (len(batch[i])):
        test_X.append(train_data.loc[batch[i][j]])
        test_y.append(test_data.loc[batch[i][j]])
    test_X = np.array(test_X)
    ones = np.ones((test_X.shape[0],1))
    test_X = np.hstack((test_X,ones))

    #retrieve the actual answers
    test_y = np.array(test_y)

    #calculate prediction
    prediction = test_X.dot(b)

    #make the attual answer and prediction 
    test_y = test_y.flatten()
    prediction = prediction.flatten()

# sum(np.array(np.dot(test_X, b) > 0.5) == y_groundtruth) / len(y_groundtruth)


    #like in class if the prediction < 0, it is 5, if prediction > 0, it is 6
    correct_5 = 0
    correct_6 = 0
    incorrect =0
    fpr_5 = 0
    fpr_6 = 0
 

    for j in range(len(prediction)):
        # print (j)
        if prediction[j] > 0 and test_y[j]==6:
          correct_6+=1
            
        elif prediction[j] < 0 and test_y[j]==5:
            correct_5+=1
            
            #fpr
        elif prediction[j] < 0 and test_y[j]==6:
            fpr_5+=1 

        elif prediction[j] > 0 and test_y[j]==5:
            fpr_6+=1         
        else:
            incorrect +=1
            # pass
    correct_count=(correct_5+correct_6)
    incorrect =(fpr_5+fpr_6)
    
    print ("iteration", i+1,)
    # print("prediction",prediction)
    print("accuracy:",correct_count/len(test_y))
    print("Correct",correct_count)
    print("Incorrect",incorrect)
    print("TP",correct_5)
    print("TN",correct_6)
    # print("tpr5",fpr_5)
    # print("tpr6",fpr_6)
    print("FP",fpr_5)
    print("FN",fpr_6,"\n" )
    

""
RMSE = (np.sum((np.dot(X, b) - y)**2)/n)**0.5
print('rmse',RMSE)



iteration 1
accuracy: 0.9117647058823529
Correct 31
Incorrect 3
TP 13
TN 18
FP 1
FN 2 

iteration 2
accuracy: 0.9117647058823529
Correct 31
Incorrect 3
TP 15
TN 16
FP 2
FN 1 

iteration 3
accuracy: 0.8529411764705882
Correct 29
Incorrect 5
TP 17
TN 12
FP 1
FN 4 

iteration 4
accuracy: 0.8823529411764706
Correct 30
Incorrect 4
TP 10
TN 20
FP 0
FN 4 

iteration 5
accuracy: 0.8529411764705882
Correct 29
Incorrect 5
TP 12
TN 17
FP 1
FN 4 

iteration 6
accuracy: 0.9090909090909091
Correct 30
Incorrect 3
TP 13
TN 17
FP 0
FN 3 

iteration 7
accuracy: 0.9090909090909091
Correct 30
Incorrect 3
TP 12
TN 18
FP 0
FN 3 

iteration 8
accuracy: 0.9090909090909091
Correct 30
Incorrect 3
TP 12
TN 18
FP 1
FN 2 

iteration 9
accuracy: 0.9090909090909091
Correct 30
Incorrect 3
TP 15
TN 15
FP 0
FN 3 

iteration 10
accuracy: 0.9393939393939394
Correct 31
Incorrect 2
TP 15
TN 16
FP 1
FN 1 

rmse 0.26916125134909286
